In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
from google.colab import files
from PIL import Image
import numpy as np
from numpy import asarray 
import pandas as pd
import torchvision
from torchvision import transforms
import os

In [ ]:
# cnn binary

from keras.models import load_model
binary_model = load_model('/content/gdrive/My Drive/riskmap_urop_data/cnn_damage.hdf5')
print(binary_model)

In [ ]:
# cnn 3 category (normal)

normal_model = load_model('/content/gdrive/My Drive/riskmap_urop/cnnmodel.hdf5')
print(normal_model)

In [ ]:
# test images

from keras.preprocessing.image import img_to_array, load_img
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
from matplotlib.pyplot import imshow
%matplotlib inline

cnn_results = []

def img_show(img):

  img = load_img(img)
  imgplot = plt.imshow(img)
  plt.show()

def img_prediction(img, model, results):

  img = load_img(img)
  img = img_to_array(img)
  img = np.expand_dims(img, axis=0)
  
  img = tf.cast(img, tf.float32)
  img = (img/127.5) - 1
  img = tf.image.resize(img, [256, 256])

  pred = model.predict(img)
  results.append(pred)
 
  # damage_dict = {}
  # damage_dict['No damage'] = pred[0][0]
  # damage_dict['Mild damage'] = pred[0][1]
  # damage_dict['Severe damage'] = pred[0][2]

  # for key, value in damage_dict.items():
  #   print(key, ": ", value)
  # print("Classification: ", max(damage_dict, key = damage_dict.get))

In [ ]:
# get results
 
!unzip -uq '/content/gdrive/My Drive/riskmap_urop/puerto_rico_test'
path = os.path.expanduser('puerto_rico_test/')

import glob
images = glob.glob(path + '/*.jpg')

# binary

# model is binary_model
# results is binary_results
binary_results = []

for img in images:
  img_prediction(img, binary_model, binary_results)

# normal

# model is cnn_model
# results is cnn_results
normal_results = []

for img in images:
  img_prediction(img, normal_model, normal_results)

In [ ]:
# num_images = len(rf_results)
i = 0
binary_weight = 1
normal_weight = 1

truth = ["severe", "none", "severe", "none", "severe", "mild", "severe", "severe", "severe", "none"]
correct = 0

for img in images:
  img_show(img)
  binary = binary_results[i]
  normal = normal_results[i]
  none = (binary[0][0]*binary_weight + normal[0][0]*normal_weight)/2
  mild = (binary[0][2]*binary_weight + normal[0][1]*normal_weight)/2
  severe = (binary[0][2]*binary_weight + normal[0][2]*normal_weight)/2
  if none > mild and none > severe:
    result = "none"
  elif mild > severe:
    result = "mild"
  else: 
    result = "severe"
  print("Classification:", result)
  if result == truth[i]:
    correct += 1
  print("None:", none)
  print("Mild:", mild)
  print("Severe:", severe)
  i += 1

print()
print("Percentage Correct:", correct/0.1, "%")

Classification: severe
None: 0.010808763094246387
Mild: 0.5288341715931892
Severe: 0.9528705477714539
Classification: none
None: 0.788516640663147
Mild: 0.192634551320225
Severe: 0.02333031641319394
Classification: severe
None: 0.295333968475461
Mild: 0.5142117105424404
Severe: 0.6648329347372055
Classification: none
None: 0.7465288192033768
Mild: 0.22606128110783175
Severe: 0.027813364111352712
Classification: severe
None: 0.01825191220268607
Mild: 0.5042821895331144
Severe: 0.9658937454223633
Classification: none
None: 0.53940025344491
Mild: 0.26555923774139956
Severe: 0.19579173281090334
Classification: none
None: 0.8755742311477661
Mild: 0.12230760621605441
Severe: 0.12377564958296716
Classification: severe
None: 0.005906411213800311
Mild: 0.4995107327122241
Severe: 0.9913747012615204
Classification: severe
None: 0.012519171228632331
Mild: 0.545195534825325
Severe: 0.939061313867569
Classification: none
None: 0.8104947805404663
Mild: 0.179617256988422
Severe: 0.009933408480719663



In [ ]:
# get results

import glob

!unzip -uq '/content/gdrive/My Drive/riskmap_urop/none'
path = os.path.expanduser('all_none/')
none_images = glob.glob(path + '/*.jpg')

!unzip -uq '/content/gdrive/My Drive/riskmap_urop/mild'
path = os.path.expanduser('all_mild/')
mild_images = glob.glob(path + '/*.jpg')

!unzip -uq '/content/gdrive/My Drive/riskmap_urop/severe'
path = os.path.expanduser('all_severe/')
severe_images = glob.glob(path + '/*.jpg')

num_test_images = 100

images = none_images[:num_test_images] + mild_images[:num_test_images] + severe_images[:num_test_images]

# binary

# model is binary_model
# results is binary_results
binary_results = []

for img in images:
  img_prediction(img, binary_model, binary_results)

# normal

# model is cnn_model
# results is cnn_results
normal_results = []

for img in images:
  img_prediction(img, normal_model, normal_results)

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


In [ ]:
# num_images = len(rf_results)
i = 0
binary_weight = 1
normal_weight = 1

correct = 0

for img in images:
  # img_show(img)
  binary = binary_results[i]
  normal = normal_results[i]
  none = (binary[0][0]*binary_weight + normal[0][0]*normal_weight)/2
  mild = (binary[0][2]*binary_weight + normal[0][1]*normal_weight)/2
  severe = (binary[0][2]*binary_weight + normal[0][2]*normal_weight)/2
  if none > mild and none > severe:
    result = "none"
  elif mild > severe:
    result = "mild"
  else: 
    result = "severe"
  # print("Classification:", result)
  if i <= num_test_images and result == "none":
    correct += 1
  if i > num_test_images and i <= num_test_images*2 and result == "mild":
    correct += 1
  if i > num_test_images*2 and result == "severe":
    correct += 1
  # print("None:", none)
  # print("Mild:", mild)
  # print("Severe:", severe)
  i += 1

# print()
print("Percentage Correct:", 100*correct/len(images), "%")


Percentage Correct: 76.0 %
